In [1]:
import os
import csv 
import pandas as pd
import librosa
import numpy as np

pathname=[]
filename=[]
mfccsfeatures = []
for root, dirs, files in os.walk(r"C:\Users\Chandhinii.B\Desktop\Electrical engineering course\Winter semester 2022\Ferdinand steinbeis institute\AI\Dataset"): 
    for fname in files:
        path = os.path.join(root, fname)
        pathname.append(path)
        audio, sample_rate = librosa.load(path)
        mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
        mfccsfeatures.append(mfccs_scaled_features)
for i in pathname:
    Name = os.path.basename(i)
    filename.append(Name)    
columns=['Pathname', 'Filename','MFCCfeatures']            
df = pd.DataFrame(list(zip(pathname, filename,mfccsfeatures)),
                 columns=columns)
df.loc[df['Filename'].str.contains("_niO_"), "Class"]="leak present"
df.loc[df['Filename'].str.contains("_iO_"), "Class"]="no leak present"
df.to_csv('Final.csv')

In [2]:
### Split the dataset into independent and dependent dataset
X=np.array(df['MFCCfeatures'].tolist())
y=np.array(df['Class'].tolist())

In [3]:
X.shape

(1920, 40)

In [4]:
y

array(['no leak present', 'no leak present', 'no leak present', ...,
       'leak present', 'leak present', 'leak present'], dtype='<U15')

In [5]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
import tensorflow as tf
import sklearn
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [6]:
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [7]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [8]:
X_train.shape

(1536, 40)

In [9]:
y_train.shape

(1536, 2)

In [10]:
X_test.shape

(384, 40)

In [11]:
y_test.shape

(384, 2)

In [12]:
import tensorflow as tf
print(tf.__version__)

2.9.1


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [14]:
### No of classes
num_labels=y.shape[1]

In [15]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               2

In [17]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [18]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)


Epoch 1/100
48/48 [==============================] - 1s 3ms/step - loss: 14.5328 - accuracy: 0.5163
Epoch 2/100
48/48 [==============================] - 0s 3ms/step - loss: 6.7164 - accuracy: 0.5065
Epoch 3/100
48/48 [==============================] - 0s 3ms/step - loss: 4.1121 - accuracy: 0.5247
Epoch 4/100
48/48 [==============================] - 0s 3ms/step - loss: 2.5524 - accuracy: 0.5169
Epoch 5/100
48/48 [==============================] - 0s 3ms/step - loss: 1.7695 - accuracy: 0.5085
Epoch 6/100
48/48 [==============================] - 0s 3ms/step - loss: 1.3539 - accuracy: 0.5065
Epoch 7/100
48/48 [==============================] - 0s 3ms/step - loss: 1.1691 - accuracy: 0.5072
Epoch 8/100
48/48 [==============================] - 0s 3ms/step - loss: 1.0200 - accuracy: 0.4948
Epoch 9/100
48/48 [==============================] - 0s 3ms/step - loss: 0.9484 - accuracy: 0.5117
Epoch 10/100
48/48 [==============================] - 0s 3ms/step - loss: 0.8250 - accuracy: 0.5273
Epoch 11

48/48 [==============================] - 0s 3ms/step - loss: 0.6983 - accuracy: 0.5202
Epoch 33/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6949 - accuracy: 0.5195
Epoch 34/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6985 - accuracy: 0.5215
Epoch 35/100
48/48 [==============================] - 0s 3ms/step - loss: 0.7074 - accuracy: 0.5156
Epoch 36/100
48/48 [==============================] - 0s 3ms/step - loss: 0.7034 - accuracy: 0.5221
Epoch 37/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6958 - accuracy: 0.5189
Epoch 38/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6998 - accuracy: 0.5169
Epoch 39/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6887 - accuracy: 0.5358
Epoch 40/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6979 - accuracy: 0.5228
Epoch 41/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6936 - accuracy: 0.5456
Epoch 42/100


48/48 [==============================] - 0s 3ms/step - loss: 0.6432 - accuracy: 0.6152
Epoch 65/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6310 - accuracy: 0.6165
Epoch 66/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6285 - accuracy: 0.6139
Epoch 67/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6380 - accuracy: 0.6081
Epoch 68/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6202 - accuracy: 0.6296
Epoch 69/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6204 - accuracy: 0.6315
Epoch 70/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6171 - accuracy: 0.6055
Epoch 71/100
48/48 [==============================] - 0s 3ms/step - loss: 0.6030 - accuracy: 0.6491
Epoch 72/100
48/48 [==============================] - 0s 3ms/step - loss: 0.5920 - accuracy: 0.6628
Epoch 73/100
48/48 [==============================] - 0s 2ms/step - loss: 0.5882 - accuracy: 0.6504
Epoch 74/100


48/48 [==============================] - 0s 3ms/step - loss: 0.4306 - accuracy: 0.7852
Epoch 96/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4321 - accuracy: 0.7923
Epoch 97/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4256 - accuracy: 0.7871
Epoch 98/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4304 - accuracy: 0.7891
Epoch 99/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4102 - accuracy: 0.8066
Epoch 100/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4089 - accuracy: 0.7956
Training completed in time:  0:00:14.920610


In [19]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8489583134651184
